In [1]:
__version__ = '0.3'
__author__  = "Robert Matern (r.matern@stud.uni-hannover.de)"
__date__    = ''
__url__     = ''
__copyright__ = "(C) 2015 Robert Matern"

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt

#Dynamisches Programm

Die normale Modellformulierung des Auftragsannahmeproblems im Revenue Management von Instandhaltungsprozessen:

$$V(\textbf{c}, t) = \sum_{j \in \mathcal{J}}p_{j}(t)\max[V(\textbf{c}, t-1), r_{j} + V(\textbf{c}-\textbf{a}_j, t-1)] + p_{0}(t)V(\textbf{c}, t-1) $$

$$= V(\textbf{c}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t) \max[r_j - V(\textbf{c}, t-1) + V(\textbf{c}-\textbf{a}_j, t-1), 0]$$

Erweiterung um einen Lagerbestand an Fertigerzeugnissen in Form von Produkten $j \in \mathcal{J}$:


$$V(\textbf{c}, \textbf{y}, t) = \sum_{j \in \mathcal{J}}p_{j}(t)\max[V(\textbf{c}, \textbf{y}, t-1), r_{j} + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1), r_{j} + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1), V(\textbf{c}-\textbf{a}_j, \textbf{y}+\textbf{s}_j, t-1)] + p_{0}(t)V(\textbf{c}, \textbf{y}, t-1) $$

$$= \sum_{j \in \mathcal{J}}p_{j}(t)V(\textbf{c}, \textbf{y}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1),0] +  \sum_{j \in \mathcal{J}}p_{j}(t)\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1),0] +\sum_{j \in \mathcal{J}}p_{j}(t)\max[V(\textbf{c}-\textbf{a}_j, \textbf{y}+\textbf{s}_j, t-1) - V(\textbf{c}, \textbf{y}, t-1),0] + p_{0}(t)V(\textbf{c}, \textbf{y}, t-1) $$

$$= \sum_{j \in \mathcal{J}}p_{j}(t)V(\textbf{c}, \textbf{y}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)[\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1),0] + \max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1),0]] + \max[V(\textbf{c}-\textbf{a}_j, \textbf{y}+\textbf{s}_j, t-1) - V(\textbf{c}, \textbf{y}, t-1) ,0]] + p_{0}(t)V(\textbf{c}, \textbf{y}, t-1) $$

$$= V(\textbf{c}, \textbf{y}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)[\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1),0] + \max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1),0] + \max[V(\textbf{c}-\textbf{a}_j, \textbf{y}+\textbf{s}_j, t-1) - V(\textbf{c}, \textbf{y}, t-1) ,0]]$$

## Algorithmus

In [3]:
solutions = {}

def DP_Storage(solutions, conditions, products, resources, capacities, consumtions, times, revenues, probs, stocks, shifts, max_stocks):
    '''Berechnung der Erwartungswerte des Auftragsannahmeproblems inkl. Lagerhaltung.'''
    
    # Leere Wert für den Erwartungswert des aktuellen Systemzustands.
    value = 0
    
    #Aktueller Systemzustand wird generiert.
    condition = np.append(capacities[1:], stocks[1:])
    condition = np.append(condition, times[0])
    
    #Aktueller Systemzustand wird in dem 'np.array' aller möglichen Systemzustände gesucht.
    state = np.where((conditions == condition).all(axis=1))[0][0]
    
    # Memofunktion: Die Rechnung wird nur fortgeführt, sofern es nicht schon berechnet wurde.
    if state not in solutions:
                
        # Sofern es sich nicht um einen Endknoten des Entscheidungsbaums handelt,
        # werden folgende Schritte eingeleitet:
        if times[0]!=0:
            # Der Erwartungswert t-1 ohne Akeptanz wird ermittelt und im Wert 'reject' gespeichert.
            reject = DP_Storage(solutions, conditions, products, resources, capacities, consumtions, times[1:], revenues, probs, stocks, shifts, max_stocks)
            
            # Für den Erwartungswert t-1 mit Akzeptanz werden Numpy-Arrays in der Länge der Anzahl an Produkten erstellt.
            accept = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            oc = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            
            # Da hier auch ein Lagerhaltungbestand berücksichtigt ist, wird auch hierführ Numpy-Arrays erstellt. 
            accept_stock = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            oc_stock = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order_stock = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            
            # Da hier auch ein Lagerhaltungbestand berücksichtigt ist, wird auch hierführ Numpy-Arrays erstellt. 
            accept_storage = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            oc_storage = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order_storage = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            
            # For-Schleife über alle Produkte, sofern die Kapazitäten keinen negativen Werte annehmen.
            for j in products[1:]:
                    
                change = capacities-consumtions[j]
                decrease_stock = stocks-shifts[j]
                increase_stock = stocks+shifts[j]
                
                if np.all((change) >= np.float32(0)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    accept_j = DP_Storage(solutions, conditions, products, resources, change, consumtions, times[1:], revenues, probs, stocks, shifts, max_stocks)
                    oc[j-1] = reject-accept_j
                    max_order[j-1] = max(revenues[j]-reject+accept_j, 0)
                    accept[j-1] = probs[j][times[0]]*max_order[j-1]            
                
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept[j-1] = np.float32(0)
                
                if np.all((decrease_stock) >= np.float32(0)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    accept_stock_j = DP_Storage(solutions, conditions, products, resources, capacities, consumtions, times[1:], revenues, probs, decrease_stock, shifts, max_stocks)
                    oc_stock[j-1] = reject-accept_stock_j
                    max_order_stock[j-1] = max(revenues[j]-reject+accept_stock_j, 0)
                    accept_stock[j-1] = probs[j][times[0]]*max_order_stock[j-1]
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept_stock[j-1] = np.float32(0)
                
                if np.all((change) >= np.float32(0)) and np.all((increase_stock) <= (max_stocks)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    accept_storage_j = DP_Storage(solutions, conditions, products, resources, change, consumtions, times[1:], revenues, probs, increase_stock, shifts, max_stocks)
                    oc_storage[j-1] = reject-accept_storage_j
                    max_order_storage[j-1] = max(accept_storage_j-reject, 0)
                    accept_storage[j-1] = probs[j][times[0]]*max_order_storage[j-1]
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept_storage[j-1] = np.float32(0)


                            
            # Summierung der Erwartungswerte ohne und mit Akzeptanz.
            value = np.float32(reject + np.sum(accept) + np.sum(accept_stock) + np.sum(accept_storage))
            # Für den aktuellen Systemzustand wird der Ertragswert und die opt. Politik in das Dict "solutions" gespeichert.
            opt_order = max(max_order)
            opt_order_stock = max(max_order_stock)
            opt_order_storage = max(max_order_storage)
            if opt_order == np.float32(0) and opt_order_stock == np.float32(0) and opt_order_storage == np.float32(0):
                solutions[state] = [conditions[state], np.float32(0), 0, 'KA', 0]
                print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2], solutions[state][3]
            elif opt_order >= opt_order_stock:
                order = np.where((max_order == opt_order))[0][0]
                solutions[state] = [conditions[state], value, order+1, 'AA', revenues[order+1]-oc[order]]
                print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2], solutions[state][3]
            elif opt_order_stock >= opt_order_storage:
                order_stock = np.where((max_order_stock == opt_order_stock))[0][0]
                solutions[state] = [conditions[state], value, order_stock+1, 'LE', revenues[order_stock+1]-oc_stock[order_stock]]
                print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2],  solutions[state][3]
            elif opt_order_storage > opt_order_stock and opt_order_storage > opt_order:
                order_storage = np.where((max_order_storage == opt_order_storage))[0][0]
                solutions[state] = [conditions[state], value, order_storage+1, 'LP', revenues[order_storage+1]-oc_storage[order_storage]]
                print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2],  solutions[state][3]
            else:
                solutions[state] = [conditions[state], np.float32(0), 0, 'KA', 0]
                print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2], solutions[state][3]
        # Sofern es sich um einen Endknoten des Entscheidungsbaums handelt, werden folgende Schritte eingeleitet:
        else:
            # Erwartungswert enspricht der Grenzbedingung V(c,0)=0, für n >= 0.
            value = np.float32(0)
            # Ein Endzustand wird mit einem Erwartungswert 0 in das Dict "solutions" gespeichert.
            solutions[state] = [conditions[state], value, 0, 'KA', 0]
            print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2], solutions[state][3]
            #print 'Periode:', times[0], 'Res.Kapa:', capacities[1:], '#Lösungen:', len(solutions), 'Lösung:', value         
            
    # Memofunktion: Sofern das Ergebnis breits berechnet wurde, wird der Wert aus dem Dict "solutions" verwendet.
    else:
        value = solutions[state][1]
               
    return value

## Erstellung der Struktur als NetworkX-Graph

In [4]:
import networkx as nx

def Structure_Storage(solutions, conditions, products, resources, consumtions, revenues, stocks, shifts, max_stocks):
    '''Generierung der Stuktur der Problemstellung.'''
    
    # MultiDiGraph erstellen.
    graph=nx.MultiDiGraph()
    
    # Knoten für alle Lösungen (solutions) erstellen.
    for key in solutions.iterkeys():
        graph.add_node(key, label=solutions[key][0], value=solutions[key][1],
                   capacity=solutions[key][0][:(len(resources)-1)], time=solutions[key][0][-1],
                      stock=solutions[key][0][(len(resources)-1):-1])
    
    # Endknoten zur Nutzung der NetworkX-Algorithmen anlegen.
    graph.add_node("end", label='end', value=0,
                   capacity=0, stock=0, time=0)
    
    # Kanten erstellen.
    # Schleife über alle Lösungen.
    for i in solutions.iterkeys():
        # Handelt es sich um einen Zeitpunkt 0, dann wird der Systemzustand mit dem 'end'-Knoten verbunden.
        if solutions[i][0][-1] == 0:
            graph.add_edge(i, "end", key=0, modus=0, weight=0, revenue=0, time=0)
        # Sonst führe die Schleife aus.
        else:
            # Aufgrund der differenzierten Auftragsannahme erfolgt eine Schleife über alle Produkte.
            for j in products:
                
                # Kapazitätsveränderung aufgrund der Anfragen nach Produkt 'j' wird erfasst.
                change = solutions[i][0][:len(resources)-1]-consumtions[j][1:]
                # Keine negativen Kapazitätsbestände möglich.
                if np.all((change) >= np.float32(0)): 
                    # Zielzustand wird ermittelt. 
                    reduction = np.append(change, solutions[i][0][len(resources)-1:-1])
                    reduction = np.append(reduction, solutions[i][0][-1]-1)
                    state = np.where((conditions == reduction).all(axis=1))[0][0]
                    # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                    no_reduction = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                    state2 = np.where((conditions == no_reduction).all(axis=1))[0][0]
                    # Prüfung und Verknüpfung der Anfragen     
                    if j == 0:
                        graph.add_edge(i, state, key=j, modus='KA', label='KA', weight=revenues[j]-solutions[state2][1]+solutions[state][1], weight_goal=solutions[state][1], revenue=revenues[j], goal=solutions[state][0], time=solutions[i][0][-1])
                    else:
                        # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                        if revenues[j] >= solutions[state2][1]-solutions[state][1]:
                            graph.add_edge(i, state, key=j, modus='AA', label=str(j)+'-AA', weight=revenues[j]-solutions[state2][1]+solutions[state][1], weight_goal=solutions[state][1], revenue=revenues[j], goal=solutions[state][0], time=solutions[i][0][-1])
                        else:
                            graph.add_edge(i, state, key=j, modus='KA', label=str(j)+'-KA', weight=0, weight_goal=solutions[state][1], revenue=0, goal=solutions[state][0], time=solutions[i][0][-1])
                
                # Lagerveränderung aufgrund der Anfragen nach Produkt 'j' wird erfasst.
                decrease_stock = solutions[i][0][(len(resources)-1):-1]-shifts[j][1:]
                if np.all((decrease_stock) >= np.float32(0)): 
                    # Zielzustand wird ermittelt. 
                    reduction_stock = np.append(solutions[i][0][:len(resources)-1], decrease_stock)
                    reduction_stock = np.append(reduction_stock, solutions[i][0][-1]-1)
                    state_stock = np.where((conditions == reduction_stock).all(axis=1))[0][0]
                    # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                    no_reduction_stock = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                    state2_stock = np.where((conditions == no_reduction_stock).all(axis=1))[0][0]
                    # Prüfung und Verknüpfung der Anfragen     
                    if j == 0:
                        graph.add_edge(i, state_stock, key=j, modus='KA', label='KA', weight=revenues[j]-solutions[state2_stock][1]+solutions[state_stock][1], weight_goal=solutions[state_stock][1], revenue=revenues[j], goal=solutions[state_stock][0], time=solutions[i][0][-1])
                    else:
                        # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                        if revenues[j] >= solutions[state2_stock][1]-solutions[state_stock][1]:
                            graph.add_edge(i, state_stock, key=j, modus='LE', label=str(j)+'-LE', weight=revenues[j]-solutions[state2_stock][1]+solutions[state_stock][1], weight_goal=solutions[state_stock][1], revenue=revenues[j], goal=solutions[state_stock][0], time=solutions[i][0][-1])
                        else:
                            graph.add_edge(i, state_stock, key=j, modus='KA', label=str(j)+'-KA', weight=0, weight_goal=solutions[state_stock][1], revenue=0, goal=solutions[state_stock][0], time=solutions[i][0][-1])
                
                # Lagererhöhung aufgrund der Anfragen nach Produkt 'j' wird erfasst.
                increase_stock = solutions[i][0][(len(resources)-1):-1]+shifts[j][1:]
                if np.all((change) >= np.float32(0)) and np.all((increase_stock) <= (max_stocks[1:])): 
                    # Zielzustand wird ermittelt. 
                    propagation_stock = np.append(change, increase_stock)
                    propagation_stock = np.append(propagation_stock, solutions[i][0][-1]-1)
                    state_storage = np.where((conditions == propagation_stock).all(axis=1))[0][0]
                    # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                    no_reduction_storage = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                    state2_storage = np.where((conditions == no_reduction_storage).all(axis=1))[0][0]
                    # Prüfung und Verknüpfung der Anfragen
                    if j == 0:
                        graph.add_edge(i, state_storage, key=j, modus='KA', label='KA', weight=revenues[j]-solutions[state2_storage][1]+solutions[state_storage][1], weight_goal=solutions[state_storage][1], revenue=revenues[j], goal=solutions[state_storage][0], time=solutions[i][0][-1])
                    else:
                        # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                        if revenues[j] >= solutions[state2_storage][1]-solutions[state_storage][1]:
                            graph.add_edge(i, state_storage, key=j, modus='LP', label=str(j)+'-LP', weight=solutions[state_storage][1]-solutions[state2_storage][1], weight_goal=solutions[state_storage][1], revenue=0, goal=solutions[state_storage][0], time=solutions[i][0][-1])
                        else:
                            graph.add_edge(i, state_storage, key=j, modus='KA', label=str(j)+'-KA', weight=0, weight_goal=solutions[state_storage][1], revenue=0, goal=solutions[state_storage][0], time=solutions[i][0][-1])
                
    return graph

KeyboardInterrupt: 

## Ermittlung der optimalen Politik

In [ ]:
def Best_Politic_Stock(graph, times, resources, stocks, solutions):
    '''Ermittlung der optimalen Politik ohne Berücksichtigung der Nachfrage.'''
    
    print 'Optimalen Politik zum Zeitpunkt "t" und unter Beachtung der Restkapazitäten "c[h]":', '\n'
        
    # List mit topologischer Sortierung des Graphen wird erstellt.
    list = nx.topological_sort(graph)
    
    # Leere Listen für die 'Pandas'-Tabellenspalten erstellen
    db_sol = []
    db_time = []
    db_res = []
    db_sto = []
    db_suc = []
    db_bp = []
    db_mo = []
    db_oc = []
    #db_sucp = []
     
    # Schleife über alle Knoten der topologisch sortierten Liste.
    for node in list:
        # Listen für 'Pandas'-Tabellenspalten erstellen.
        db_sol.append(graph.node[node]['value'])
        db_time.append(graph.node[node]['time'])
        db_suc.append(graph.successors(node))
        # Ist die Schleife zum Endknoten der Liste gekommen, dann wird sie unterbrochen.
        if node == 'end':
            db_bp.append(0)
            db_mo.append('-')
            db_oc.append(0)
            #db_sucp.append(0)
            break
        # Ist die Schleife zu einem Endzeitpunkt gekommen, dann wird sie übersprungen.
        elif graph.node[node]['time'] == 0:
            db_bp.append(0)
            db_mo.append('-')
            db_oc.append(0)
            #db_sucp.append(0)
            continue
        # Sonst werden die möglichen Auftragsannahmen des Knotens in eine Liste geschrieben.
        else:
            db_bp.append(solutions[node][2])
            db_mo.append(solutions[node][3])
            db_oc.append(solutions[node][4])
            #db_sucp.append(graph.successors(node)[solutions[node][3]-1])
        
        for suc in graph.successors(node):
            for order in graph.edge[node][suc].iterkeys():
                # Boolescher Wert 'not_best_politic' zur Ermittlung des Pfads wird angelegt.
                if order == solutions[node][2] and graph.edge[node][suc][order]['modus'] == solutions[node][3]:
                    graph.edge[node][suc][order]['not_best_politic']=False
                    graph.edge[node][suc][order]['style']='bold'
                else:
                    graph.edge[node][suc][order]['not_best_politic']=True
            
    
    # Spalten für die Ressourcenkapazitäten erstellen.
    for res in resources[1:]:
        db_res.append('$c_{'+str(res)+'}$')
    db_cap = []
    for c in range(len(db_res)):
        cap = []
        for n in list:
            if n == 'end':
                cap.append(0)
            else:
                cap.append(graph.node[n]['capacity'][c])
        db_cap.append(cap)

    # Spalten für die Lagerbestände erstellen.
    for sto in range(len(stocks[1:])):
        db_sto.append('$y_{'+str(sto+1)+'}$')
    db_stos = []
    for s in range(len(db_sto)):
        stos = []
        for n in list:
            if n == 'end':
                stos.append(0)
            else:
                stos.append(graph.node[n]['stock'][s])
        db_stos.append(stos)

    # Tabelle erstellen                
    db = pd.DataFrame(index=list)
    for i, res in enumerate(db_res):
        db[res] = db_cap[i]
    for k, sto in enumerate(db_sto):
        db[sto] = db_stos[k]
    db['$t$'] = db_time
    db['ExpValue'] = db_sol
    db['Successor'] = db_suc
    db['$j^*$'] = db_bp
    db['$m$'] = db_mo
    #db['Best Successor'] = db_sucp
    db['$r_{j}-OC_{j}$'] = db_oc
    
    
    # Tabelle ausgeben.
    db = db[:-1]
    db.to_csv('Table_Graph.csv')
    print db, '\n'
    
    # Optimaler Pfad wird mittels Dijkstra-Algorithmus geschrieben.
    path = nx.dijkstra_path(graph, source=list[0], target=list[-1], weight='not_best_politic')[:-1]
    
    # Tabelle erstellen.
    rev = [0] * len(times)
    dec = [0] * len(times)
    fr = [0] * len(times)
    to = [0] * len(times)
    for i, node in enumerate(path[:-1]):
        for order in graph.edge[node][path[i+1]].iterkeys():
            if graph.edge[node][path[i+1]][order]['not_best_politic'] == False:
                rev[graph.node[node]['time']] = graph.edge[node][path[i+1]][order]['revenue']
                dec[graph.node[node]['time']] = order
                fr[graph.node[node]['time']] = path[i]
                to[graph.node[node]['time']] = path[i+1]

    df = pd.DataFrame(dec[::-1], index=times, columns = ['Opt. Politic'])
    #df.index.name = 'time'
    df['from'] = fr[::-1]
    df['to'] = to[::-1]
    df['Revenue'] = rev[::-1]
    df['Cum_Rev'] = df.Revenue.cumsum()
    df = df[:-1]
    df.to_csv('Table_Politic.csv')
    print df, '\n'
        
    return db, df